In [2]:
import wrapper as wp
from importlib import reload
import os
reload(wp)
import pandas as pd
import numpy as np
import re

# Etiquetado usando Stanford Tagger

In [3]:
data = pd.read_csv('/media/NAS/Tesis/NER/data/CNB_data/JusticIA_DatosTranscripciones.csv')
texts = data[data['MetodoTexto'] == 'manual']['Texto'].tolist()

In [4]:
res = wp.make_prediction(texts)

500
export CLASSPATH=$CLASSPATH:/media/NAS/Tesis/NER/Stanford_NLP/stanford-corenlp-4.3.1/* && java -Xmx5g edu.stanford.nlp.pipeline.StanfordCoreNLP -props spanish -filelist list.txt


[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator mwt
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator pos
[main] INFO edu.stanford.nlp.tagger.maxent.MaxentTagger - Loading POS tagger from edu/stanford/nlp/models/pos-tagger/spanish-ud.tagger ... done [0.3 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
[main] INFO edu.stanford.nlp.sequences.SeqClassifierFlags - sutime.language=spanish
[main] INFO edu.stanford.nlp.ie.AbstractSequenceClassifier - Loading classifier from edu/stanford/nlp/models/ner/spanish.ancora.distsim.s512.crf.ser.gz ... done [0.5 sec].
[main] INFO edu.stanford.nlp.time.JollyDayHolidays - Initializing JollyDayHoliday for SUTime from classpath edu/stanf

500
export CLASSPATH=$CLASSPATH:/media/NAS/Tesis/NER/Stanford_NLP/stanford-corenlp-4.3.1/* && java -Xmx5g edu.stanford.nlp.pipeline.StanfordCoreNLP -props spanish -filelist list.txt


[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator mwt
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator pos
[main] INFO edu.stanford.nlp.tagger.maxent.MaxentTagger - Loading POS tagger from edu/stanford/nlp/models/pos-tagger/spanish-ud.tagger ... done [0.3 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
[main] INFO edu.stanford.nlp.sequences.SeqClassifierFlags - sutime.language=spanish
[main] INFO edu.stanford.nlp.ie.AbstractSequenceClassifier - Loading classifier from edu/stanford/nlp/models/ner/spanish.ancora.distsim.s512.crf.ser.gz ... done [0.5 sec].
[main] INFO edu.stanford.nlp.time.JollyDayHolidays - Initializing JollyDayHoliday for SUTime from classpath edu/stanf

0
continue due to size cero


# Etiquetado usando las palabras de las listas

In [5]:
def clean(w):
    aux = ''
    for c in w.strip().lower():
        if c == ')' or c == '(':
            continue
        aux += c
    return aux

In [6]:
def get_inter(frase, voc, tol=0.2):
    cont = 0
    if type(frase) != list:
        frase = frase.split()
    for w in frase:
        if w in voc:
            cont += 1

    if cont >= int(len(frase)*tol)+1:
        return True
    return False


def get_frase(class_stanford):
    return class_stanford['Text'].strip().lower()


def text_tagger(text, voc, tol=0.2, window=5):
    text = text.split()
    labels = []
    for i in range(window//2, len(text)):
        if get_inter(text[i-window//2:i+window//2+1], voc, tol):
            labels.append(1)
        else:
            labels.append(0)

    return labels

In [7]:
civ = pd.read_csv('/media/NAS/Tesis/NER/data/CNB_data/civilservants.csv')['CIVILSERVANTS '].tolist()
org = pd.read_csv('/media/NAS/Tesis/NER/data/CNB_data/organizations.csv')['ORGANIZATIONS '].tolist()
pla = pd.read_csv('/media/NAS/Tesis/NER/data/CNB_data/places.csv')['PLACES '].tolist()
pro = pd.read_csv('/media/NAS/Tesis/NER/data/CNB_data/prosecuted.csv')['PROSECUTED '].tolist()

In [8]:
civ = list(set([clean(w) for w in civ]))
civ = list(set([w for word in civ for w in word.split() ]))
org = list(set([clean(w) for w in org]))
org = list(set([w for word in org for w in word.split() ]))
pla = list(set([clean(w) for w in pla]))
pla = list(set([w for word in pla for w in word.split() ]))
pro = list(set([clean(w) for w in pro]))
pro = list(set([w for word in pro for w in word.split() ]))

In [9]:
data = {
    'civ':civ,
    'org':org,
    'pla':pla,
    'pro':pro
}

In [10]:
text_cats = {}

In [14]:
res_tag = []
for text in texts:
    aux = {}
    aux['Text'] = text.split()
    for k in data:
        lab = []
        for w in aux['Text']:
            if w in stop:
                lab.append('O')
                continue
            if w in data[k]:
                lab.append(k)
            else:
                lab.append('O')
        aux[k] = lab
    res_tag.append(aux)

In [15]:
for ind in range(len(res_tag)):
    res_tag[ind]['Stanford'] = ['O']*len(res_tag[ind]['civ'])

In [16]:
for r in res:
    word_list = r['Text'].split()
    for ind_w in range(len(texts[r['File']].split())):
        if texts[r['File']].split()[ind_w] in stop:
            continue
        if texts[r['File']].split()[ind_w] in word_list:
            res_tag[r['File']]['Stanford'][ind_w] = r['TAG'].split(':')[0]

In [17]:
real_labs = ['civ','org','pla','pro']
pred_labs = ['Stanford']

In [18]:
ind_text = 0
for text in res_tag:
    real_lab_list = [0]*len(text[real_labs[0]])
    pred_lab_list = [0]*len(text[real_labs[0]])
    for real in real_labs:
        for ind_p in range(len(text[real])):
            if text[real][ind_p] == 'O':
                continue
            real_lab_list[ind_p] = 1
    for pred in pred_labs:
        for ind_p in range(len(text[pred])):
            if text[pred][ind_p] == 'O':
                continue
            pred_lab_list[ind_p] = 1
    res_tag[ind_text]['real_lab'] = real_lab_list
    res_tag[ind_text]['pred_lab'] = pred_lab_list
    ind_text += 1

In [19]:
real = []
preds = []

for ind in range(len(res_tag)):
    real += res_tag[ind]['real_lab']
    preds += res_tag[ind]['pred_lab']

In [20]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

In [21]:
f1_score(real, preds)

0.18795386586928664

In [22]:
accuracy_score(real, preds)

0.8274892010599296

In [23]:
precision_score(real, preds)

0.16127849864379445

In [24]:
recall_score(real, preds)

0.22520217013000307

In [11]:
from nltk.corpus import stopwords

In [12]:
stop = stopwords.words('spanish')

In [13]:
all_texts = " ".join([t for t in texts])

# Ventana 5, tolerancia 20

In [44]:
res = {}
for k in data:
    res[k] = [0, 0] + text_tagger(all_texts, data[k], tol=0.2, window=5)

## Coincidencias con civ

In [45]:
voc_etiq = []
for i in range(len(res['civ'])):
    aux = False
    for k in res:
        if res[k][i] == 1:
            aux = True
            voc_etiq.append(1)
            break
    if aux:
        aux = False
    else:
        voc_etiq.append(0)


Métricas

In [48]:
accuracy_score(voc_etiq, preds)

0.3236959599259501

In [49]:
f1_score(voc_etiq, preds)

0.15068149701417696

Número de predicciones

In [50]:
np.sum(voc_etiq)

74107

In [51]:
np.sum(preds)

13641

## Coincidencias con org

In [57]:
voc_etiq = []
for i in range(len(res['org'])):
    aux = False
    for k in res:
        if res[k][i] == 1:
            aux = True
            voc_etiq.append(1)
            break
    if aux:
        aux = False
    else:
        voc_etiq.append(0)


Métricas

In [58]:
accuracy_score(voc_etiq, preds)

0.3236959599259501

In [59]:
f1_score(voc_etiq, preds)

0.15068149701417696

Número de predicciones

In [60]:
np.sum(voc_etiq)

74107

In [61]:
np.sum(preds)

13641

## Coincidencias con pla

In [1]:
voc_etiq = []
for i in range(len(res['pla'])):
    aux = False
    for k in res:
        if res[k][i] == 1:
            aux = True
            voc_etiq.append(1)
            break
    if aux:
        aux = False
    else:
        voc_etiq.append(0)


NameError: name 'res' is not defined

Métricas

In [63]:
accuracy_score(voc_etiq, preds)

0.3236959599259501

In [64]:
f1_score(voc_etiq, preds)

0.15068149701417696

Número de predicciones

In [65]:
np.sum(voc_etiq)

74107

In [66]:
np.sum(preds)

13641

## Coincidencias con pro

In [67]:
voc_etiq = []
for i in range(len(res['pro'])):
    aux = False
    for k in res:
        if res[k][i] == 1:
            aux = True
            voc_etiq.append(1)
            break
    if aux:
        aux = False
    else:
        voc_etiq.append(0)


Métricas

In [68]:
accuracy_score(voc_etiq, preds)

0.3236959599259501

In [69]:
f1_score(voc_etiq, preds)

0.15068149701417696

Número de predicciones

In [70]:
np.sum(voc_etiq)

74107

In [71]:
np.sum(preds)

13641

# Ventana 3, tolerancia 20

In [75]:
res = {}
for k in data:
    res[k] = [0] + text_tagger(all_texts, data[k], tol=0.2, window=3)

## Coincidencias con civ

In [76]:
voc_etiq = []
for i in range(len(res['civ'])):
    aux = False
    for k in res:
        if res[k][i] == 1:
            aux = True
            voc_etiq.append(1)
            break
    if aux:
        aux = False
    else:
        voc_etiq.append(0)


Métricas

In [77]:
accuracy_score(voc_etiq, preds)

0.25595302914806345

In [78]:
f1_score(voc_etiq, preds)

0.1664277508362054

Número de predicciones

In [79]:
np.sum(voc_etiq)

84720

In [80]:
np.sum(preds)

13641

## Coincidencias con org

In [81]:
voc_etiq = []
for i in range(len(res['org'])):
    aux = False
    for k in res:
        if res[k][i] == 1:
            aux = True
            voc_etiq.append(1)
            break
    if aux:
        aux = False
    else:
        voc_etiq.append(0)


Métricas

In [82]:
accuracy_score(voc_etiq, preds)

0.25595302914806345

In [83]:
f1_score(voc_etiq, preds)

0.1664277508362054

Número de predicciones

In [84]:
np.sum(voc_etiq)

84720

In [85]:
np.sum(preds)

13641

## Coincidencias con pla

In [86]:
voc_etiq = []
for i in range(len(res['pla'])):
    aux = False
    for k in res:
        if res[k][i] == 1:
            aux = True
            voc_etiq.append(1)
            break
    if aux:
        aux = False
    else:
        voc_etiq.append(0)


Métricas

In [87]:
accuracy_score(voc_etiq, preds)

0.25595302914806345

In [88]:
f1_score(voc_etiq, preds)

0.1664277508362054

Número de predicciones

In [89]:
np.sum(voc_etiq)

84720

In [90]:
np.sum(preds)

13641

## Coincidencias con pro

In [91]:
voc_etiq = []
for i in range(len(res['pro'])):
    aux = False
    for k in res:
        if res[k][i] == 1:
            aux = True
            voc_etiq.append(1)
            break
    if aux:
        aux = False
    else:
        voc_etiq.append(0)


Métricas

In [92]:
accuracy_score(voc_etiq, preds)

0.25595302914806345

In [93]:
f1_score(voc_etiq, preds)

0.1664277508362054

Número de predicciones

In [94]:
np.sum(voc_etiq)

84720

In [95]:
np.sum(preds)

13641

# Ventana 7, tolerancia 20

In [96]:
res = {}
for k in data:
    res[k] = [0, 0, 0] + text_tagger(all_texts, data[k], tol=0.2, window=7)

## Coincidencias con civ

In [97]:
voc_etiq = []
for i in range(len(res['civ'])):
    aux = False
    for k in res:
        if res[k][i] == 1:
            aux = True
            voc_etiq.append(1)
            break
    if aux:
        aux = False
    else:
        voc_etiq.append(0)


Métricas

In [98]:
accuracy_score(voc_etiq, preds)

0.23497223129696176

In [99]:
f1_score(voc_etiq, preds)

0.17862953905511658

Número de predicciones

In [100]:
np.sum(voc_etiq)

88996

In [101]:
np.sum(preds)

13641

## Coincidencias con org

In [102]:
voc_etiq = []
for i in range(len(res['org'])):
    aux = False
    for k in res:
        if res[k][i] == 1:
            aux = True
            voc_etiq.append(1)
            break
    if aux:
        aux = False
    else:
        voc_etiq.append(0)


Métricas

In [103]:
accuracy_score(voc_etiq, preds)

0.23497223129696176

In [104]:
f1_score(voc_etiq, preds)

0.17862953905511658

Número de predicciones

In [105]:
np.sum(voc_etiq)

88996

In [106]:
np.sum(preds)

13641

## Coincidencias con pla

In [107]:
voc_etiq = []
for i in range(len(res['pla'])):
    aux = False
    for k in res:
        if res[k][i] == 1:
            aux = True
            voc_etiq.append(1)
            break
    if aux:
        aux = False
    else:
        voc_etiq.append(0)


Métricas

In [108]:
accuracy_score(voc_etiq, preds)

0.23497223129696176

In [109]:
f1_score(voc_etiq, preds)

0.17862953905511658

Número de predicciones

In [110]:
np.sum(voc_etiq)

88996

In [111]:
np.sum(preds)

13641

## Coincidencias con pro

In [112]:
voc_etiq = []
for i in range(len(res['pro'])):
    aux = False
    for k in res:
        if res[k][i] == 1:
            aux = True
            voc_etiq.append(1)
            break
    if aux:
        aux = False
    else:
        voc_etiq.append(0)


Métricas

In [113]:
accuracy_score(voc_etiq, preds)

0.23497223129696176

In [114]:
f1_score(voc_etiq, preds)

0.17862953905511658

Número de predicciones

In [115]:
np.sum(voc_etiq)

88996

In [116]:
np.sum(preds)

13641

# Ventana 3, tolerancia 50

In [42]:
res = {}
for k in data:
    res[k] = [0] + text_tagger(all_texts, data[k], tol=0.5, window=3)

## Coincidencias con civ

In [43]:
voc_etiq = []
for i in range(len(res['civ'])):
    aux = False
    for k in res:
        if res[k][i] == 1:
            aux = True
            voc_etiq.append(1)
            break
    if aux:
        aux = False
    else:
        voc_etiq.append(0)


Métricas

In [44]:
accuracy_score(voc_etiq, preds)

0.5504374024465498

In [45]:
f1_score(voc_etiq, preds)

0.11097552221663916

In [46]:
from sklearn.metrics import precision_score, classification_report

In [47]:
print(classification_report(voc_etiq, preds))

              precision    recall  f1-score   support

           0       0.60      0.85      0.70     68113
           1       0.23      0.07      0.11     42083

    accuracy                           0.55    110196
   macro avg       0.41      0.46      0.41    110196
weighted avg       0.46      0.55      0.47    110196



In [37]:
set(voc_etiq), set(preds)

({0, 1}, {0, 1})

Número de predicciones

In [32]:
len(all_texts.split())

110196

In [29]:
np.sum(voc_etiq)

7274

In [30]:
np.sum(preds)

13641

## Coincidencias con org

In [123]:
voc_etiq = []
for i in range(len(res['org'])):
    aux = False
    for k in res:
        if res[k][i] == 1:
            aux = True
            voc_etiq.append(1)
            break
    if aux:
        aux = False
    else:
        voc_etiq.append(0)


Métricas

In [124]:
accuracy_score(voc_etiq, preds)

0.5504374024465498

In [125]:
f1_score(voc_etiq, preds)

0.11097552221663916

Número de predicciones

In [126]:
np.sum(voc_etiq)

42083

In [127]:
np.sum(preds)

13641

## Coincidencias con pla

In [128]:
voc_etiq = []
for i in range(len(res['pla'])):
    aux = False
    for k in res:
        if res[k][i] == 1:
            aux = True
            voc_etiq.append(1)
            break
    if aux:
        aux = False
    else:
        voc_etiq.append(0)


Métricas

In [129]:
accuracy_score(voc_etiq, preds)

0.5504374024465498

In [130]:
f1_score(voc_etiq, preds)

0.11097552221663916

Número de predicciones

In [131]:
np.sum(voc_etiq)

42083

In [132]:
np.sum(preds)

13641

## Coincidencias con pro

In [133]:
voc_etiq = []
for i in range(len(res['pro'])):
    aux = False
    for k in res:
        if res[k][i] == 1:
            aux = True
            voc_etiq.append(1)
            break
    if aux:
        aux = False
    else:
        voc_etiq.append(0)


Métricas

In [134]:
accuracy_score(voc_etiq, preds)

0.5504374024465498

In [135]:
f1_score(voc_etiq, preds)

0.11097552221663916

Número de predicciones

In [136]:
np.sum(voc_etiq)

42083

In [137]:
np.sum(preds)

13641

corrido de tolerancia de 0.2 a 0.8, saltos de (0.1) y ventana de 3 a 11, salto de 2, grafica con f1, presicion, recuerdo y soporte

con la mejor configuracion sacar la nube de palabra de lo que se etiquetó de las dos mejores categorias y ver cual tiene la mejor eficacia y cual con mayores errores (que si y no etiqueta)